In [45]:
import pandas as pd

In [46]:
patient_df = pd.read_csv(r"D:\DataScience\Datascience Assignment\04 Patient_Data.csv")
billing_df = pd.read_csv(r"D:\DataScience\Datascience Assignment\04 Billing_Data.csv")

In [47]:

# Show Sumary

In [48]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [49]:
# select columns Relevant for Billing
# Require Columns : ['PatientId','Deparment','Doctor','BillAmount']

In [50]:
billing_columns = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_billing_df = patient_df[billing_columns]

In [51]:
patient_billing_df.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [52]:
# Drop Administrative Columns
# Columns to Drop : ['ReceptionistID','CheckInTime']

In [53]:
patient_df = patient_df.drop(['ReceptionistID','CheckInTime'],axis = 1)
patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [54]:
# Total Bill Amount per Department(groupby)
deparment_bill = patient_df.groupby('Department')['BillAmount'].sum()
deparment_bill

Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64

In [55]:
# Remove Duplicate Patient Records

In [56]:
patient_df = patient_df.drop_duplicates(subset = 'PatientID')
patient_df.shape

(5, 5)

In [57]:
# Fill Missing BillAmount with Mean

In [58]:
mean_bill = patient_df['BillAmount'].mean()
patient_df['BillAmount'] = patient_df['BillAmount'].fillna(mean_bill)

In [59]:
# Merge Billing & Patient Datasets
# Merge on PatientID

In [60]:
merged_df  = pd.merge(
    patient_df,
    billing_df,
    on = 'PatientID',
    how='inner'
    )
merged_df.head

<bound method NDFrame.head of    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  >

In [61]:
# Concatenate New Patient Data
# New patients for current week
new_patients = pd.DataFrame({
    'PatientID' :[501,502],
    'Department':['Cardiology','Orthopedics'],
    'Doctor':['Dr.Sharma','Dr.Mehta'],
    'BillAmount':[15000 , 18000]
    })

In [62]:
# Row - wise concatenation
merged_df = pd.concat([merged_df , new_patients] , axis=0)
merged_df.tail()

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0
0,501,NaN,Cardiology,Dr.Sharma,15000.000000,NaN,NaN
1,502,NaN,Orthopedics,Dr.Mehta,18000.000000,NaN,NaN


In [63]:
#  Concatenate New Billing Columns(Column- wise)
# New columns: ['InsuranceCovered','FinalAmount']

In [64]:
new_columns = pd.DataFrame({
    'InsuranceCovered':['Yes'] * len(merged_df),
    'FinalAmount': merged_df['BillAmount'] * 0.9
    })

In [65]:
# Column-wise concatenation
final_df = pd.concat([merged_df.reset_index(drop=True),
                      new_columns.reset_index(drop = True)],
                     axis=1)

In [66]:
final_df.head()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000.0,3000.0,Yes,4500.0
1,102,Bob,Neurology,Dr. John,6233.333333,1500.0,3500.0,Yes,5610.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0,Yes,6750.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0,Yes,5580.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0,Yes,5610.0
